## Segmenting and Clustering 3-1

##### The cell below will install all the dependencies that we will need.

In [55]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

!pip install bs4 # beautifulsoup

print('Folium installed')
print('beautifulsoup installed')
print('Libraries imported.')

Folium installed
beautifulsoup installed
Libraries imported.


In [56]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import pandas as pd
import requests
from bs4 import BeautifulSoup

##### The below url contains an html table with the required data 

In [57]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

##### get the contents of the webpage in text format and store in a variable called html_data

In [58]:
html_data  = requests.get(url).text

Using beautiful soup extract the table with GameStop Quarterly Revenue and store it into a dataframe named gme_revenue

In [59]:
soup = BeautifulSoup(html_data,"html5lib")

In [60]:
tables = soup.find_all('table')

In [61]:
for index,table in enumerate(tables):
    if ("Postal Code" in str(table)):
        table_index = index
print(table_index)

0


#### Creating The dataframe consist of three columns: PostalCode, Borough, and Neighborhood. 

In [62]:
canada_data = pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        code = col[0].text.strip("\n")
        borough = col[1].text.strip("\n")
        neighbourhood = col[2].text.strip("\n")
        canada_data = canada_data.append({"Postal Code":code, "Borough":borough, "Neighbourhood":neighbourhood}, ignore_index=True)
    
canada_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Extracting Only process the cells that have an assigned borough.

In [63]:
canada_data.drop(canada_data.loc[canada_data['Borough']=='Not assigned'].index, inplace=True)

In [64]:
canada_data.reset_index(drop=True, inplace=True)
canada_data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [65]:
# Shape of the Dataframe
canada_data.shape

(103, 3)

#### Extracting geographical coordinates of each postal code into a Pandas DataFrame

In [70]:
df_geo = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Creating neighborhoods Dataframe by merging Canada_data and df_geo DataFrames 

In [71]:
neighborhoods = pd.merge(canada_data, df_geo, on='Postal Code')

In [72]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Decided to Extract the Borough North York to visualize neighborhoods

In [73]:
York_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
York_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


#### Using geopy library to get the latitude and longitude values of North York

In [74]:
address = 'North York, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


#### Create a map of North York with neighborhoods superimposed on top.

In [75]:
# create map of North York using latitude and longitude values
map_Yoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(York_data['Latitude'], York_data['Longitude'], York_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Yoke)  
    
map_Yoke